### How long should people infected with the virus isolate for? Possible steps to answer this are:
- Find all terms that describe isolation (incubation, quarantine, isolation, infectious, transmissable, spreading, transferable, contagious)
- Look for all numbers used near these words (in the form 'XX days') that describe the recommend isolation period.
- Put all numbers into a list and average them to determine the recommended isolation period.
- Spot check a few examples to check it's working as expected.

In [ ]:
def recommended_isolation(data, keywords, step=5, day_threshold=25):
    days=[]
    row_num=[]
    for indx, row in data.iterrows():
        days_regex = '[0-9]+'
        words = row['text'].split()
        for index, word in enumerate(words):
            if index < len(words) - 1:
                next_word = words[index+1]
            else:
                next_word = ''
            if index >= step:
                words_left = words[index-step:index]
            else:
                words_left = []
            words_right = words[index+1:index+step+1]
            surrounding_words = words_left + words_right
            if re.search(days_regex, word) and next_word == 'days' and any(word in keywords for word in surrounding_words):
                days.append(word)
                row_num.append(indx)
    
    valid_days=[]
    for day in days:
        try: 
            float(day)
        except:
            continue
        else:
            if float(day) < day_threshold:
                valid_days.append(float(day))
                
    return np.asarray(valid_days).mean()

In [ ]:
keywords = ['isolation', 'incubation', 'quarantine', 'infectious', 'transmissable', 'spreading', 'transferable', 'contagious']
days_to_isolate = recommended_isolation(data_eng[:1000], keywords)

In [ ]:
print(days_to_isolate)

7.512121212121213


### Output

#### Preliminary checks
Spot checks showed some estimates of isolation periods were sensible, however others were not being used in the correct context. For example, the paper may mention one of the keywords but not be stating that it is a recommended isolation period, rather just using the terms in a different context. 

#### Possible next steps
Find better way to determine context and other characteristics that are used in the sentences we're looking for. May be best to return to this later rather than investing too much time on only one element of the research.